In [1]:
import pandas as pd
import numpy as np

from IPython.display import display

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [2]:
landscape = dataset1[0].loc['10μM'].tolist()

In [5]:
greedy_count = 0
total_count = 1000
for i in range(0,total_count):
    results = simulate(landscape, timesteps=1200, seed='0001')
    if results['greedy_path'] == results['actual_path']:
        greedy_count += 1
greedy_count / total_count

0.81

In [6]:
results = simulate([landscape], timesteps=700, seed='0001')
plot_simulation(results)

In [8]:
results['greedy_path']

['0001', '1001', '1011', '1111', '1110']

In [9]:
results['actual_path']

['0001', '1001', '1011', '1111', '1110']

In [22]:
landscape = dataset2.loc['AMC'].tolist()
landscape2 = dataset2.loc['AM'].tolist()
for i in range (0, 1000):
    results = simulate([landscape, landscape2], durations=[200, 1000])
    if results['T_1'] != -1:
        plot_simulation(results)

In [14]:
print(landscape)

[0.1843, 0.2975, 0.9245, 0.0, 0.2433, 0.2245, 1.2084, 1.0891, 0.2879, 0.388, 1.0567, 0.6425, 0.3103, 0.3669, 1.2874, 1.1547]


In [13]:
print(landscape2)

[1.3978, 1.2749, 1.2275, 0.0, 1.3698, 1.3747, 1.3969, 1.2193, 1.1188, 1.1841, 1.3063, 1.0, 1.273, 1.2823, 1.4499, 1.2504]
